In [1]:
from keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import layers
from keras import models
import tensorflow as tf

/Users/lucasremigio/Developer/SSC/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import os

# Caminho local para a pasta raiz do projeto
root_path = "./"  # ou o caminho absoluto, ex: "C:/Users/teu_usuario/Documents/SSC-AI-GARBAGE-DETECTION"

# Listar diretórios no caminho raiz
print("📁 Diretórios no caminho raiz:")
print(os.listdir(root_path))

# Verificar conteúdo de um caminho específico
specific_path = os.path.join(root_path, "garbage-noaug-70-15-15")
if os.path.exists(specific_path):
    print(f"\n📁 Conteúdo de {specific_path}:")
    print(os.listdir(specific_path))
else:
    print(f"\n❌ Caminho {specific_path} não existe")

# Função para listar diretórios com profundidade
def list_dirs(path, indent=0):
    for item in os.listdir(path):
        full_path = os.path.join(path, item)
        if os.path.isdir(full_path):
            print(" " * indent + "📁 " + item)
            if indent < 4:
                list_dirs(full_path, indent + 2)
        else:
            print(" " * indent + "📄 " + item)

# Explorar estrutura de diretórios
print("\n📂 Estrutura de diretórios:")
list_dirs(root_path, 0)

📁 Diretórios no caminho raiz:
['garbage-dataset-aug-87-6-6', '.DS_Store', 'requirements.txt', 'garbage-dataset', 'garbage-noaug-70-15-15', '.gitignore', '.venv', 'ssc.py', '.git', 'SSC.ipynb']

📁 Conteúdo de ./garbage-noaug-70-15-15:
['README.roboflow.txt', 'valid', 'README.dataset.txt', '.DS_Store', 'test', 'train']

📂 Estrutura de diretórios:
📁 garbage-dataset-aug-87-6-6
  📄 README.roboflow.txt
  📁 valid
    📁 paper
    📁 clothes
    📁 metal
    📁 cardboard
    📁 trash
    📁 glass
    📁 biological
    📁 battery
    📁 plastic
    📁 shoes
  📄 README.dataset.txt
  📁 test
    📁 paper
    📁 clothes
    📁 metal
    📁 cardboard
    📁 trash
    📁 glass
    📁 biological
    📁 battery
    📁 plastic
    📁 shoes
  📁 train
    📁 paper
    📁 clothes
    📁 metal
    📁 cardboard
    📁 trash
    📁 glass
    📁 biological
    📁 battery
    📁 plastic
    📁 shoes
📄 .DS_Store
📄 requirements.txt
📁 garbage-dataset
  📁 paper
    📄 paper_926.jpg
    📄 paper_2718.jpg
    📄 paper_1205.jpg
    📄 paper_1563.jpg
 

In [4]:
# Improved Metal GPU detection for Apple Silicon
try:
    # First try looking for GPU devices (newer TF versions label Metal as GPU)
    gpus = tf.config.list_physical_devices('GPU')
    if len(gpus) > 0:
        print(f"Found {len(gpus)} GPU device(s)")
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU acceleration enabled (Metal)")
    # If no GPU found, try looking specifically for MPS devices
    elif hasattr(tf.config, 'list_physical_devices') and len(tf.config.list_physical_devices('MPS')) > 0:
        mps_devices = tf.config.list_physical_devices('MPS')
        tf.config.experimental.set_visible_devices(mps_devices[0], 'MPS')
        print("MPS (Metal) device enabled")
    else:
        print("No GPU or MPS device found, using CPU")
        
    # Verify what device is being used
    print("\nDevice being used:", tf.config.get_visible_devices())
    
    # Test with a simple operation to confirm GPU usage
    with tf.device('/GPU:0'):
        a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        b = tf.constant([[5.0, 6.0], [7.0, 8.0]])
        c = tf.matmul(a, b)
        print("Matrix multiplication result:", c)
        print("GPU test successful!")
except Exception as e:
    print(f"Error setting up GPU: {e}")
    print("Falling back to CPU")

Found 1 GPU device(s)
GPU acceleration enabled (Metal)

Device being used: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Matrix multiplication result: tf.Tensor(
[[19. 22.]
 [43. 50.]], shape=(2, 2), dtype=float32)
GPU test successful!


2025-05-31 15:37:57.412200: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-05-31 15:37:57.412251: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-05-31 15:37:57.412260: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-05-31 15:37:57.412314: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-31 15:37:57.412335: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
train_dir = specific_path + "/train"
validation_dir = specific_path + "/valid"
test_dir = specific_path + "/test"

# Images are 640, but 224 is way faster for training
IMG_SIZE = 160

train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32
)

validation_dataset = image_dataset_from_directory(
    validation_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32
)

test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32
)



Found 13833 files belonging to 10 classes.
Found 2965 files belonging to 10 classes.
Found 2964 files belonging to 10 classes.


In [6]:
train_dataset = train_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)


In [7]:
for data_batch, labels_batch in train_dataset:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (32, 160, 160, 3)
labels batch shape: (32,)


2025-05-31 15:39:18.103871: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [8]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
# Dropout for better generalization
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation="relu")(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
    metrics=['accuracy'])



In [17]:
# Add to your training cell
import time

class TimeoutCallback(tf.keras.callbacks.Callback):
    def __init__(self, max_time_hrs=2):
        super().__init__()
        self.max_time_sec = max_time_hrs * 3600
        self.start_time = time.time()
        
    def on_epoch_end(self, epoch, logs=None):
        elapsed = time.time() - self.start_time
        if elapsed > self.max_time_sec:
            print(f"\nReached time limit ({self.max_time_sec/3600:.1f}h). Stopping training.")
            self.model.stop_training = True

# Maximum 2 hours of training to prevent overheating
timeout_cb = TimeoutCallback(max_time_hrs=1)


In [19]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    patience=5,
    restore_best_weights=True
)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "model_checkpoint.h5", 
    save_best_only=True,
    monitor="val_accuracy"
)

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=[early_stopping, timeout_cb, checkpoint_cb]
)

Epoch 1/30
432/433 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6369 - loss: 1.1168

433/433 ━━━━━━━━━━━━━━━━━━━━ 23s 53ms/step - accuracy: 0.6369 - loss: 1.1168 - val_accuracy: 0.6030 - val_loss: 1.2385
Epoch 2/30
432/433 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6471 - loss: 1.1099

433/433 ━━━━━━━━━━━━━━━━━━━━ 23s 53ms/step - accuracy: 0.6471 - loss: 1.1100 - val_accuracy: 0.6411 - val_loss: 1.1505
Epoch 3/30
432/433 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6469 - loss: 1.1242

433/433 ━━━━━━━━━━━━━━━━━━━━ 23s 53ms/step - accuracy: 0.6469 - loss: 1.1244 - val_accuracy: 0.6516 - val_loss: 1.1878
Epoch 4/30
433/433 ━━━━━━━━━━━━━━━━━━━━ 23s 53ms/step - accuracy: 0.6464 - loss: 1.1688 - val_accuracy: 0.6462 - val_loss: 1.2666
Epoch 5/30
433/433 ━━━━━━━━━━━━━━━━━━━━ 23s 53ms/step - accuracy: 0.6472 - loss: 1.2659 - val_accuracy: 0.6347 - val_loss: 1.3532
Epoch 6/30
433/433 ━━━━━━━━━━━━━━━━━━━━ 23s 54ms/step - accuracy: 0.6324 - loss: 1.4319 - val_accuracy: 0.6034 - val_loss: 1.6786
Epoch 7/30
433/433 ━━━━━━━━━━━━━━━━━━━━ 24s 55ms/step - accuracy: 0.6313 - loss: 1.6911 - val_accuracy: 0.6192 - val_loss: 1.9984
Epoch 8/30
433/433 ━━━━━━━━━━━━━━━━━━━━ 26s 60ms/step - accuracy: 0.6231 - loss: 2.0345 - val_accuracy: 0.6044 - val_loss: 2.4129


In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo' , label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()